<a href="https://colab.research.google.com/github/arutraj/.githubcl/blob/main/6_Implementing_SMS_spam_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Table of Contents
### 1. Loading the Dataset
### 2. Pre-processing the Dataset
### 3. Feature Engineering and Model Building
> ##### a. Creating Meta Features
> ##### b. Counting Nouns and Verbs
> ##### c. Model Building for Meta Features
> ##### d. Tf-Idf Features
> ##### e. Model Building for Complete Feature Set

## 1. Loading the dataset

In [1]:
# Import libraries
import pandas as pd
import string

In [2]:
!unzip /content/archive_SMSSpam.zip

Archive:  /content/archive_SMSSpam.zip
  inflating: spam.csv                


In [3]:
!wc -l /content/spam.csv

5572 /content/spam.csv


In [ ]:
# Mounting the drive
from google.colab import drive
drive.mount('/content/drive')

In [5]:
# Load dataset
data = pd.read_csv('/content/spam.csv', encoding='iso-8859-1')
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [7]:
# distribution of class
data['v1'].value_counts(normalize=True)

,proportion
v1,
ham,0.865937
spam,0.134063


## 2. Pre-processing the dataset

In [10]:
# Sample document: lowercase
cleaned = data['v2'][2].lower()

In [11]:
# Sample document
cleaned

"free entry in 2 a wkly comp to win fa cup final tkts 21st may 2005. text fa to 87121 to receive entry question(std txt rate)t&c's apply 08452810075over18's"

In [12]:
# Pre-initialised list of punctuations
punctuations = string.punctuation

In [13]:
# Pre-initialised list of punctuations
punctuations

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [14]:
# Sample document: Remove punctuations
cleaned = "".join(character for character in cleaned if character not in punctuations)

In [15]:
# Sample document
cleaned

'free entry in 2 a wkly comp to win fa cup final tkts 21st may 2005 text fa to 87121 to receive entry questionstd txt ratetcs apply 08452810075over18s'

In [16]:
# Import spacy
from spacy.lang.en import English

# Load English tokenizer, tagger, parser, NER and word vectors
nlp = English()

In [17]:
# spacy document
my_doc = nlp(cleaned)

In [18]:
# Create list of word tokens
token_list = []
for token in my_doc:
    token_list.append(token.text)

In [19]:
# Import stop words
from spacy.lang.en.stop_words import STOP_WORDS

In [20]:
# Create list of word tokens after removing stopwords
filtered_sentence =[]

# Iterate over the tokens
for word in token_list:
    # Get token text
    lexeme = nlp.vocab[word]
    # Check if stopword or not
    if lexeme.is_stop == False:
        filtered_sentence.append(word)

# Print tokens and filtered sentence
print(token_list)
print(filtered_sentence)
cleaned = filtered_sentence

['free', 'entry', 'in', '2', 'a', 'wkly', 'comp', 'to', 'win', 'fa', 'cup', 'final', 'tkts', '21st', 'may', '2005', 'text', 'fa', 'to', '87121', 'to', 'receive', 'entry', 'questionstd', 'txt', 'ratetcs', 'apply', '08452810075over18s']
['free', 'entry', '2', 'wkly', 'comp', 'win', 'fa', 'cup', 'final', 'tkts', '21st', '2005', 'text', 'fa', '87121', 'receive', 'entry', 'questionstd', 'txt', 'ratetcs', 'apply', '08452810075over18s']


In [21]:
# Joining the tokenised words in sample document
cleaned = " ".join(cleaned)
cleaned

'free entry 2 wkly comp win fa cup final tkts 21st 2005 text fa 87121 receive entry questionstd txt ratetcs apply 08452810075over18s'

In [22]:
# Preprocessing function
def clean_text(text):
    ## lower case
    cleaned = text.lower()

    ## remove punctuations
    punctuations = string.punctuation
    cleaned = "".join(character for character in cleaned if character not in punctuations)

    ## remove stopwords
    my_doc = nlp(cleaned)
    token_list = []
    for token in my_doc:
        token_list.append(token.text)

    ## remove stop words
    filtered_sentence =[]

    for word in token_list:
        lexeme = nlp.vocab[word]
        if lexeme.is_stop == False:
            filtered_sentence.append(word)

    ## Store cleaned document
    cleaned = filtered_sentence
    cleaned = " ".join(cleaned)

    return cleaned

In [24]:
# Applying the preprocessin function
data["cleaned"] = data["v2"].apply(lambda x : clean_text(x))
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4,cleaned
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN,jurong point crazy available bugis n great wor...
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN,ok lar joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN,u dun early hor u c
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN,nah nt think goes usf lives


## 3. Feature Engineering and Model Building

### a. Creating Meta Features

> 1. Number of words in original text
> 2. Number of words in cleaned text
> 3. Number of characters including spaces in the cleaned text
> 4. Number of characters excluding spaces in the cleaned text
> 5. Number of digits in the cleaned text

In [25]:
# Creating meta features

# Number of words in original text
data["word_count"] = data["v2"].apply(lambda x : len(x.split()))
# Number of words in cleaned text
data["word_count_cleand"] = data["cleaned"].apply(lambda x : len(x.split()))

# Number of characters including spaces in the cleaned text
data["char_count"] = data["cleaned"].apply(lambda x : len(x))
# number of characters excluding spaces in the cleaned text
data["char_count_without_spaces"] = data["cleaned"].apply(lambda x : len(x.replace(" ","")))

# Number of digits in the cleaned text
data["num_dig"] = data["cleaned"].apply(lambda x :  sum([1 if w.isdigit() else 0 for w in x.split()]))

In [26]:
# Print dataset
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4,cleaned,word_count,word_count_cleand,char_count,char_count_without_spaces,num_dig
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN,jurong point crazy available bugis n great wor...,20,15,79,65,0
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN,ok lar joking wif u oni,6,6,23,18,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN,free entry 2 wkly comp win fa cup final tkts 2...,28,22,131,110,3
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN,u dun early hor u c,11,6,19,14,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN,nah nt think goes usf lives,13,6,27,22,0


### b. Counting Nouns and Verbs

In [27]:
# Import spacy English language model
import spacy
nlp = spacy.load("en_core_web_sm")

In [28]:
# spacy document
document = nlp(data['cleaned'][0])

In [29]:
# Sample document
document

jurong point crazy available bugis n great world la e buffet cine got amore wat

In [30]:
# POS tags
all_tags = []
for w in document:
    all_tags.append(w.tag_)

In [31]:
# POS tags
all_tags

['NNP',
 'VBP',
 'NNP',
 'JJ',
 'NNP',
 'CC',
 'JJ',
 'NN',
 'NNP',
 'NNP',
 'NNP',
 'NNP',
 'VBD',
 'NNP',
 'NN']

In [32]:
# Dictionary of noun and verb POS tags
pos_dic = {"noun" : ["NNP", "NN", "NNS", "NNPS"], "verb" : ["VBZ", "VB", "VBD","VBG", "VBN"]}

In [33]:
# Sample document: Noun count
count = 0
for tag in all_tags:
    if tag in pos_dic['noun']:
        count += 1

In [34]:
# Sample document: Noun count
count

10

In [35]:
# Function for noun and verb counts
def pos_check(txt, family):

    # spacy document
    txt = nlp(txt)

    all_tags = []

    # Get pos tag
    for w in txt:
        all_tags.append(w.tag_)

    count = 0

    # Count number of nouns and verbs
    for tag in all_tags:
        if tag in pos_dic[family]:
            count += 1

    return count

In [37]:
# Sample document: Noun count
pos_check("They are  in the ground", "verb")

0

In [38]:
# Sample document: Verb count
pos_check("They are playing in the ground", "noun")

1

In [39]:
# Applying the function
data["noun_count"] = data["cleaned"].apply(lambda x : pos_check(x, "noun"))
data["verb_count"] = data["cleaned"].apply(lambda x : pos_check(x, "verb"))

In [40]:
# Print output
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4,cleaned,word_count,word_count_cleand,char_count,char_count_without_spaces,num_dig,noun_count,verb_count
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN,jurong point crazy available bugis n great wor...,20,15,79,65,0,10,1
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN,ok lar joking wif u oni,6,6,23,18,0,3,1
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN,free entry 2 wkly comp win fa cup final tkts 2...,28,22,131,110,3,12,0
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN,u dun early hor u c,11,6,19,14,0,6,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN,nah nt think goes usf lives,13,6,27,22,0,1,1


### c. Model Building for Meta Features

In [42]:
# Label encoding target variable .. strings to int.
from sklearn.preprocessing import LabelEncoder

target = data["v1"].values
target = LabelEncoder().fit_transform(target)

In [43]:
# List of features
train = data[['word_count', 'word_count_cleand', 'char_count',
              'char_count_without_spaces', 'num_dig', 'noun_count',
              'verb_count']]

In [44]:
# Train-Validation split
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(train, target, random_state=20, stratify=target)

In [45]:
# Train and Validatio dataset
(x_train.shape, y_train.shape), (x_valid.shape, y_valid.shape)

(((4179, 7), (4179,)), ((1393, 7), (1393,)))

In [46]:
# Naive bayes
from sklearn import naive_bayes

In [47]:
# Multinomial naive bayes
model = naive_bayes.MultinomialNB()

In [48]:
# Fit model on training data
model.fit(x_train, y_train)

MultinomialNB()

In [49]:
# Prediction on training data
pred_train = model.predict(x_train)
# Prediction on validation data
pred_valid = model.predict(x_valid)

In [50]:
# Accuracy
from sklearn.metrics import accuracy_score

In [51]:
# Training accuracy
accuracy_score(y_train, pred_train)

0.9418521177315147

In [52]:
# Validation accuracy
accuracy_score(y_valid, pred_valid)

0.9368269921033741

### d. Tf-idf Features

In [53]:
# Import Tf-Idf Vectoriser
from sklearn.feature_extraction.text import TfidfVectorizer

In [54]:
# Tf-Idf Vectoriser
word_tfidf = TfidfVectorizer(max_features=500)

In [55]:
# Fit Tf-Idf Vectoriser
word_tfidf.fit(data["cleaned"].values)

TfidfVectorizer(max_features=500)

In [56]:
# Transform
word_vectors_tfidf = word_tfidf.transform(data["cleaned"].values)

In [57]:
# Tf-Idf vectors
word_vectors_tfidf

<5572x500 sparse matrix of type '<class 'numpy.float64'>'
	with 21920 stored elements in Compressed Sparse Row format>

In [60]:
# Combining meta features and Tf-Idf features
from scipy.sparse import hstack, csr_matrix

# List of meta features
meta_features = ['word_count', 'word_count_cleand',
       'char_count', 'char_count_without_spaces', 'num_dig', 'noun_count',
       'verb_count']

# Meta features
feature_set1 = data[meta_features]

# Combined features
train = hstack([word_vectors_tfidf, csr_matrix(feature_set1)], "csr")

### e. Model Building for complete feature set

In [61]:
# Train and Validation datasets
x_train, x_valid, y_train, y_valid = train_test_split(train, target, random_state=20, stratify=target)

In [62]:
# Train and Validation datasets
(x_train.shape, y_train.shape), (x_valid.shape, y_valid.shape)

(((4179, 507), (4179,)), ((1393, 507), (1393,)))

In [63]:
# Multinomial Naive Bayes Model
model = naive_bayes.MultinomialNB()

In [64]:
# Fit model
model.fit(x_train, y_train)

MultinomialNB()

In [65]:
# Predcition on Training data
pred_train = model.predict(x_train)
# Predcition on Validation data
pred_valid = model.predict(x_valid)

In [66]:
# Training accuracy
accuracy_score(y_train, pred_train)

0.9679349126585307

In [67]:
# Validation accuracy
accuracy_score(y_valid, pred_valid)

0.9583632447954056